# Consistency DANN

In [13]:
import rul_datasets
import rul_adapt
import pytorch_lightning as pl
import omegaconf

## Reproduce original configurations

You can reproduce the original experiments by Siahpour et al. by using the `get_consistency_dann` constructor function.
Known differences to the original paper are:

* the `consistency_factor` is set to 1.0 because the real value is not mentioned in the paper
* the raw vibration data of XJTU-SY is preprocessed by extracting the standard deviation from each window because the given architecture could not handle the raw data
* the target data is truncated with 80% `percent_broken` to exclude data from the point of failure

Additional `kwargs` for the trainer, e.g. `accelerator="gpu"` for training on a GPU, can be passed to the function as a dictionary.
The first dictionary is used for the pre-training trainer and the second one for the main trainer.

In [4]:
pl.seed_everything(42, workers=True)  # makes it reproducible
pre_training, main_training = rul_adapt.construct.get_consistency_dann(
    "cmapss", 3, 1, {"max_epochs": 1}, {"max_epochs": 1}
)

Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


The function returns two tuples.
The first contains everything needed for pre-training, the second everything needed for the main training.

In [8]:
pre_dm, pre_approach, pre_trainer = pre_training
pre_trainer.fit(pre_approach, pre_dm)


  | Name               | Type               | Params
----------------------------------------------------------
0 | train_loss         | MeanSquaredError   | 0     
1 | val_loss           | MeanSquaredError   | 0     
2 | _feature_extractor | CnnExtractor       | 3.3 K 
3 | _regressor         | FullyConnectedHead | 221   
----------------------------------------------------------
3.5 K     Trainable params
0         Non-trainable params
3.5 K     Total params
0.014     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


After pre-training, we can use the pre-trained networks to initialize the main training.
The networks of the pre-training approach, i.e. `feature_extractor` and `regressor`, can be accessed as properties.

In [9]:
dm, approach, domain_disc, trainer = main_training
approach.set_model(pre_approach.feature_extractor, pre_approach.regressor, domain_disc)
trainer.fit(approach, dm)
trainer.test(approach, dm)


   | Name                     | Type                  | Params
--------------------------------------------------------------------
0  | train_source_loss        | MeanSquaredError      | 0     
1  | consistency_loss         | ConsistencyLoss       | 0     
2  | val_source_rmse          | MeanSquaredError      | 0     
3  | val_target_rmse          | MeanSquaredError      | 0     
4  | val_source_score         | RULScore              | 0     
5  | val_target_score         | RULScore              | 0     
6  | test_source_rmse         | MeanSquaredError      | 0     
7  | test_target_rmse         | MeanSquaredError      | 0     
8  | test_source_score        | RULScore              | 0     
9  | test_target_score        | RULScore              | 0     
10 | _feature_extractor       | CnnExtractor          | 3.3 K 
11 | _regressor               | FullyConnectedHead    | 221   
12 | dann_loss                | DomainAdversarialLoss | 21    
13 | frozen_feature_extractor | CnnExtractor    

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0             DataLoader 1
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    test/source_rmse         83.58596801757812
    test/source_score          371773.96875
    test/target_rmse                                  84.6204833984375
    test/target_score                                   342500.28125
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/source_rmse/dataloader_idx_0': 83.58596801757812,
  'test/source_score/dataloader_idx_0': 371773.96875},
 {'test/target_rmse/dataloader_idx_1': 84.6204833984375,
  'test/target_score/dataloader_idx_1': 342500.28125}]

If you only want to see the hyperparameters, you can use the `get_consistency_dann_config` function.
This returns an `omegaconf.DictConfig` which you can modify.
Afterwards, you can pass the config to `consistency_dann_from_config` to receive the training-ready approach.

In [12]:
cmapss_three2one_config = rul_adapt.construct.get_consistency_dann_config("cmapss", 3, 1)
print(omegaconf.OmegaConf.to_yaml(cmapss_three2one_config, resolve=True))

dm:
  source:
    _target_: rul_datasets.CmapssReader
    fd: 3
    window_size: 20
  target:
    fd: 1
    percent_broken: 0.8
  kwargs:
    batch_size: 128
feature_extractor:
  _target_: rul_adapt.model.CnnExtractor
  input_channels: 14
  conv_filters:
  - 32
  - 16
  - 1
  seq_len: 20
  fc_units: 20
  conv_dropout: 0.5
  fc_dropout: 0.5
regressor:
  _target_: rul_adapt.model.FullyConnectedHead
  input_channels: 20
  act_func_on_last_layer: false
  units:
  - 10
  - 1
  dropout: 0.5
domain_disc:
  _target_: rul_adapt.model.FullyConnectedHead
  input_channels: 20
  act_func_on_last_layer: false
  units:
  - 1
consistency_pre:
  _target_: rul_adapt.approach.ConsistencyApproachPretraining
  lr: 1.0e-05
consistency:
  _target_: rul_adapt.approach.ConsistencyApproach
  consistency_factor: 1.0
  lr: 1.0e-05
  max_epochs: 3000
trainer_pre:
  _target_: pytorch_lightning.Trainer
  max_epochs: 3000
trainer:
  _target_: pytorch_lightning.Trainer
  max_epochs: 3000



## Run your own experiments

You can use the Consistency DANN implementation to run your own experiments with different hyperparameters or on different datasets.
Here we build an approach with an LSTM feature extractor.

In [15]:
source = rul_datasets.CmapssReader(3)
target = source.get_compatible(1, percent_broken=0.8)

pre_dm = rul_datasets.RulDataModule(source, batch_size=32)
dm = rul_datasets.DomainAdaptionDataModule(
    pre_dm, rul_datasets.RulDataModule(target, batch_size=32),
)

feature_extractor = rul_adapt.model.LstmExtractor(
    input_channels=14,
    lstm_units=[16],
    fc_units=8,
)
regressor = rul_adapt.model.FullyConnectedHead(
    input_channels=8,
    units=[8, 1],
    act_func_on_last_layer=False,
)
domain_disc = rul_adapt.model.FullyConnectedHead(
    input_channels=8,
    units=[8, 1],
    act_func_on_last_layer=False,
)

pre_approach = rul_adapt.approach.ConsistencyApproachPretraining(lr=0.001)
pre_approach.set_model(feature_extractor, regressor)
pre_trainer = pl.Trainer(max_epochs=1)
trainer.fit(pre_approach, pre_dm)

approach = rul_adapt.approach.ConsistencyApproach(
    consistency_factor=1.0, lr=0.001, max_epochs=1
)
approach.set_model(
    pre_approach.feature_extractor, pre_approach.regressor, domain_disc
)
trainer = pl.Trainer(max_epochs=1)
trainer.fit(approach, dm)
trainer.test(approach, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name               | Type               | Params
----------------------------------------------------------
0 | train_loss         | MeanSquaredError   | 0     
1 | val_loss           | MeanSquaredError   | 0     
2 | _feature_extractor | LstmExtractor      | 2.2 K 
3 | _regressor         | FullyConnectedHead | 81    
----------------------------------------------------------
2.3 K     Trainable params
0         Non-trainable params
2.3 K     Total params
0.009     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

   | Name                     | Type                  | Params
--------------------------------------------------------------------
0  | train_source_loss        | MeanSquaredError      | 0     
1  | consistency_loss         | ConsistencyLoss       | 0     
2  | val_source_rmse          | MeanSquaredError      | 0     
3  | val_target_rmse          | MeanSquaredError      | 0     
4  | val_source_score         | RULScore              | 0     
5  | val_target_score         | RULScore              | 0     
6  | test_source_rmse         | MeanSquaredError      | 0     
7  | test_target_rmse         | MeanSquaredError      | 0     
8  | test_source_score        | RULScore              | 0     
9  | test_target_score        | RULScore              | 0     
10 | _feature_extractor       | LstmExtract

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0             DataLoader 1
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    test/source_rmse         82.91014099121094
    test/source_score            350401.5
    test/target_rmse                                  83.95921325683594
    test/target_score                                   323265.96875
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/source_rmse/dataloader_idx_0': 82.91014099121094,
  'test/source_score/dataloader_idx_0': 350401.5},
 {'test/target_rmse/dataloader_idx_1': 83.95921325683594,
  'test/target_score/dataloader_idx_1': 323265.96875}]